# Test the three models

In [ ]:
import keras
from keras import backend as K
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, Add, Lambda
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.optimizers import Adam, RMSprop, Adadelta
from keras.applications import VGG19

import datetime
import matplotlib.pyplot as plt
import numpy as np
import scipy
import sys
import os

from data_helper import predict_15k, save_hist, save_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, History

from time import gmtime, strftime
from data_helper import readImg, readImgInv, imagePatches, removeBlackImg, removeCorrespondence, check_and_create

In [ ]:
from mapgenlib.losses_extend import dice_coef_loss, iou_loss, wasserstein_loss
from mapgenlib.res_unit import res_block, decoder, encoder, build_res_unet
from mapgenlib.discrimnator import build_discriminator_simple_dc, build_discriminator_critic
from mapgenlib.discrimnator import build_discriminator_patchgan_srgan, build_discriminator_patchgan_cycle

from functools import partial
from keras.layers.merge import _Merge
class RandomWeightedAverage(_Merge):
    """Provides a (random) weighted average between real and generated image samples"""
    """Citation: https://github.com/eriklindernoren/Keras-GAN/blob/master/wgan_gp/wgan_gp.py#L110"""
    def _merge_function(self, inputs):
        alpha = K.random_uniform((32, 1, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

In [ ]:
class GAN4MapGen(): # Based on u-net, residual u-net and pix2pix
    # Reference: https://github.com/eriklindernoren/Keras-GAN/blob/master/pix2pix/pix2pix.py
    
    def __init__(self, method, batch_size=16):

        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        #self.clip_value = 0.01

        ## Calculate output shape of D (PatchGAN) better version
        #self.patch_size = 32
        #self.nb_patches = int((self.img_rows / self.patch_size) * (self.img_cols / self.patch_size))
        #self.patch_gan_dim = (self.patch_size, self.patch_size, self.channels)
        
        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64
        self.batch_size = batch_size
        
        self.method = method
        
        if method == "dc":
            self.initial_dc()
            self.train_cell = self.train_cell_dc
            self.valid_cell = self.valid_cell_dc
            
            # Adversarial ground truths
            self.valid = np.ones((batch_size, 1))
            self.fake = np.zeros((batch_size, 1))
        
        if method == "wgangp":
            self.initial_wgangp()
            self.train_cell = self.train_cell_wgangp
            self.valid_cell = self.valid_cell_wgangp
            
            # Adversarial ground truths
            self.valid = -np.ones((batch_size, 1))
            self.fake =  np.ones((batch_size, 1))
        
        if method == "srgan":
            # Calculate output shape of D (PatchGAN)
            patch = int(self.img_rows / 2**4)
            self.disc_patch = (patch, patch, 1)
            
            self.initial_srgan()
            self.train_cell = self.train_cell_srgan
            self.valid_cell = self.valid_cell_srgan
            
            self.valid = np.ones((batch_size,) + self.disc_patch)
            self.fake = np.zeros((batch_size,) + self.disc_patch)
        
        if method == "p2p":
            # Calculate output shape of D (PatchGAN)
            patch = int(self.img_rows / 2**4)
            self.disc_patch = (patch, patch, 1)
            
            self.initial_p2p()
            self.train_cell = self.train_cell_p2p
            self.valid_cell = self.valid_cell_p2p
            
            self.valid = np.ones((batch_size,) + self.disc_patch)
            self.fake = np.zeros((batch_size,) + self.disc_patch)
        
        """
        #optimizer = Adam(0.0002, 0.5) # Original
        #optimizer = Adam(0.0001, 0.5) # Original # Latest achieved by 0.00008
        #optimizer = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08) # An old version of Pix2pix
        optimizer = Adam(0.0004)
        
        #-------------------------
        # Construct Computational
        #   Graph of Generator
        #-------------------------

        # Build the generator
        #self.generator = self.build_generator() # Old generator from 
        self.generator = self.build_res_unet_generator()

        # Input images and their conditioning images
        #img_A = Input(shape=self.img_shape) # Target
        img_B = Input(shape=self.img_shape) # Input

        # By conditioning on B generate a fake version of A
        fake_A = self.generator(img_B)
        
        # Build and compile the discriminator
        self.discriminator = build_discriminator_simple_dc(self.img_shape)
        
        self.discriminator.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
        #self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Discriminators determines validity of translated images / condition pairs
        #valid = self.discriminator([fake_A, img_B])
        #self.combined = Model(inputs=[img_A, img_B], outputs=[valid, fake_A])
        
        valid = self.discriminator(fake_A)
        self.combined = Model(inputs= img_B, outputs=[valid, fake_A])
        
        # Original Pix2Pix - low weight for discriminator
        self.combined.compile(loss=['mse', 'mae'], #['mse', 'mae'] original
                              loss_weights=[1, 100], # [1, 100] original
                              optimizer=optimizer, metrics=['accuracy'])
        """
    
    
    
    
    """ WGAN_GP """
    def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
        """
        Computes gradient penalty based on prediction and weighted real / fake samples
        """
        gradients = K.gradients(y_pred, averaged_samples)[0]
        # compute the euclidean norm by squaring ...
        gradients_sqr = K.square(gradients)
        #   ... summing over the rows ...
        gradients_sqr_sum = K.sum(gradients_sqr,
                                  axis=np.arange(1, len(gradients_sqr.shape)))
        #   ... and sqrt
        gradient_l2_norm = K.sqrt(gradients_sqr_sum)
        # compute lambda * (1 - ||grad||)^2 still for each single sample
        gradient_penalty = K.square(1 - gradient_l2_norm)
        # return the mean as loss over all the batch samples
        return K.mean(gradient_penalty)
    
    def initial_wgangp(self):
        
        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        optimizer = RMSprop(lr=0.00005)

        # Build the generator and critic
        self.generator = build_res_unet(self.img_shape)
        self.critic = build_discriminator_critic(self.img_shape)
        
        #-------------------------------
        # Construct Computational Graph
        #       for the Critic
        #-------------------------------
        
        # Freeze generator's layers while training critic
        self.generator.trainable = False
        
        # Input images and generate imgs
        src_img = Input(shape=self.img_shape)
        real_img = Input(shape=self.img_shape)
        fake_img = self.generator(src_img)
        
        # Discriminator determines validity of the real and fake images
        fake = self.critic(fake_img)
        valid = self.critic(real_img)
        
        # Construct weighted average between real and fake images
        interpolated_img = RandomWeightedAverage()([real_img, fake_img])
        # Determine validity of weighted sample
        validity_interpolated = self.critic(interpolated_img)
        
        # Use Python partial to provide loss function with additional
        # 'averaged_samples' argument
        partial_gp_loss = partial(self.gradient_penalty_loss, averaged_samples=interpolated_img)
        partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

        self.critic_model = Model(inputs=[real_img, src_img], outputs=[valid, fake, validity_interpolated])
        self.critic_model.compile(loss=[wasserstein_loss, wasserstein_loss, partial_gp_loss],
                                        optimizer=optimizer,
                                        loss_weights=[1, 1, 10])
        #-------------------------------
        # Construct Computational Graph
        #         for Generator
        #-------------------------------

        # For the generator we freeze the critic's layers
        self.critic.trainable = False
        self.generator.trainable = True

        # Generate images based of noise
        fake_img = self.generator(src_img)
        # Discriminator determines validity
        valid = self.critic(fake_img)
        # Defines generator model
        self.generator_model = Model(src_img, [valid, fake_img])
        self.generator_model.compile(loss=[wasserstein_loss, "mse"], optimizer=optimizer)
        
    def train_cell_wgangp(self, imgs_A, imgs_B, valid, fake):
        
        dummy = np.zeros((self.batch_size, 1)) # Dummy gt for gradient penalty
        
        for _ in range(self.n_critic):

            # ---------------------
            #  Train Discriminator
            # ---------------------
            # Train the critic
            d_loss = self.critic_model.train_on_batch([imgs_A, imgs_B], [valid, fake, dummy])

        # ---------------------
        #  Train Generator
        # ---------------------
        g_loss = self.generator_model.train_on_batch(imgs_B, valid)
        return d_loss, g_loss
    
    def valid_cell_wgangp(self, x_test_sim, y_test_sim):
        num_test = len(x_test_sim)
        valid_test = np.ones((num_test,1))
        t_loss = self.generator_model.evaluate(x_test_sim, [valid_test, y_test_sim], verbose=0)
        return t_loss
    """ WGAN_GP ends""" 
    
    
    """ Pix2Pix """
    def initial_p2p(self):
        
        optimizer = Adam(0.0004)
        
        # Build the generator
        self.generator = build_res_unet(self.img_shape)
        
        # Build and compile the discriminator
        self.discriminator = build_discriminator_patchgan_cycle(self.img_shape, self.df)
        #self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        self.discriminator.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
        
        # Input images and generate imgs
        img_B = Input(shape=self.img_shape)
        fake_A = self.generator(img_B)
        
        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        
        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(fake_A)
        
        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(inputs=img_B, outputs=[valid, fake_A])
        #self.combined.compile(loss=['binary_crossentropy', 'mse'], loss_weights=[1, 100], 
        #                      optimizer=optimizer, metrics=['accuracy'])
        #self.combined.compile(loss=['binary_crossentropy', 'mse'], loss_weights=[1, 10], 
        #                      optimizer=optimizer, metrics=['accuracy'])
        #self.combined.compile(loss=['binary_crossentropy', 'mse'], loss_weights=[1, 100], 
        #                      optimizer=optimizer, metrics=['accuracy'])

        self.combined.compile(loss=['mse', 'mae'], # Oroginal pix2pix
                              loss_weights=[1, 100], 
                              optimizer=optimizer, metrics=['accuracy'])
        
    def train_cell_p2p(self, imgs_A, imgs_B, valid, fake):
        # Condition on B and generate a translated version
        fake_A = self.generator.predict(imgs_B)
                
        # Train the discriminators (original images = real / generated = Fake)
        d_loss_real = self.discriminator.train_on_batch(imgs_A, valid)
        d_loss_fake = self.discriminator.train_on_batch(fake_A, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # Train the generators
        g_loss = self.combined.train_on_batch(imgs_B, [valid, imgs_A])
        return d_loss, g_loss
    
    def valid_cell_p2p(self, x_test_sim, y_test_sim):
        num_test = len(x_test_sim)
        valid_test = np.ones((num_test,) + self.disc_patch)
        t_loss = self.combined.evaluate(x_test_sim, [valid_test, y_test_sim], verbose=0)
        return t_loss
    """ Pix2Pix ends""" 
    
    
    
    """ SRGAN """
    def initial_srgan(self):
        optimizer = Adam(0.0004)
        
        # We use a pre-trained VGG19 model to extract image features from the high resolution
        # and the generated high resolution images and minimize the mse between them
        self.vgg = self.build_vgg()
        self.vgg.trainable = False
        self.vgg.compile(loss='mse',optimizer=optimizer,metrics=['accuracy'])

        # Build and compile the discriminator
        self.discriminator = build_discriminator_patchgan_srgan(self.img_shape, self.df)
        self.discriminator.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
        
        # Build the generator
        self.generator = build_res_unet(self.img_shape)
        self.generator.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

        # Input images and generate imgs
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)
        
        # Generate image
        fake_A = self.generator(img_B)
        
        # Extract image features of the generated img
        fake_features = self.vgg(fake_A)
        
        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        
        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(fake_A)
        
        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(inputs=[img_B, img_A], outputs=[valid, fake_features])
        self.combined.compile(loss=['binary_crossentropy', 'mse'], 
                              loss_weights=[1, 10], optimizer=optimizer, metrics=['mae'])
    
    def train_cell_srgan(self, imgs_A, imgs_B, valid, fake):
        # Condition on B and generate a translated version
        fake_A = self.generator.predict(imgs_B)
                
        # Train the discriminators (original images = real / generated = Fake)
        d_loss_real = self.discriminator.train_on_batch(imgs_A, valid)
        d_loss_fake = self.discriminator.train_on_batch(fake_A, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        valid = np.ones((self.batch_size,) + self.disc_patch)
        
        image_features = self.vgg.predict(imgs_A)
        
        # Train the generators
        g_loss = self.combined.train_on_batch([imgs_B, imgs_A], [valid, image_features])
        return d_loss, g_loss
    
    def valid_cell_srgan(self, x_test_sim, y_test_sim):
        num_test = len(x_test_sim)
        valid_test = np.ones((num_test,) + self.disc_patch)
        t_loss = self.generator.evaluate(x_test_sim, y_test_sim, verbose=0)
        return t_loss
    """ SRGAN ends"""
    
    
    
    
    """ DCGAN """
    def initial_dc(self):
        
        optimizer = Adam(0.0004)
        
        # Build the generator
        self.generator = build_res_unet(self.img_shape)
        
        # Build and compile the discriminator
        self.discriminator = build_discriminator_simple_dc(self.img_shape)
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
        # Input images and generate imgs
        img_B = Input(shape=self.img_shape)
        fake_A = self.generator(img_B)
        
        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        
        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(fake_A)
        
        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(inputs=img_B, outputs=[valid, fake_A])
        self.combined.compile(loss=['binary_crossentropy', 'mse'], loss_weights=[1, 100], optimizer=optimizer, metrics=['accuracy'])
        
    def train_cell_dc(self, imgs_A, imgs_B, valid, fake):
        # Condition on B and generate a translated version
        fake_A = self.generator.predict(imgs_B)
                
        # Train the discriminators (original images = real / generated = Fake)
        d_loss_real = self.discriminator.train_on_batch(imgs_A, valid)
        d_loss_fake = self.discriminator.train_on_batch(fake_A, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # Train the generators
        g_loss = self.combined.train_on_batch(imgs_B, [valid, imgs_A])
        return d_loss, g_loss
    
    def valid_cell_dc(self, x_test_sim, y_test_sim):
        num_test = len(x_test_sim)
        valid_test = np.ones((num_test,1))
        t_loss = self.combined.evaluate(x_test_sim, [valid_test, y_test_sim], verbose=0)
        return t_loss
    """ DC ends"""
    
    

    
    def train(self, x_train_sim, y_train_sim, x_test_sim, y_test_sim, outPath, epochs, sample_interval=50, patience = 5):
    
        start_time = datetime.datetime.now()
        
        valid = self.valid
        fake  = self.fake
        
        total_samples = len(x_train_sim)
        ids = np.arange(total_samples)
        np.random.shuffle(ids)
        n_batches = int(total_samples / self.batch_size)
        
        train_acc = []
        train_loss = []
        valid_acc = []
        valid_loss = []

        for epoch in range(epochs):
            for batch_i, (imgs_A, imgs_B) in enumerate(load_batch(x_train_sim, y_train_sim, self.batch_size)):
            
                d_loss, g_loss = self.train_cell(imgs_A, imgs_B, valid, fake)
            
                if batch_i % sample_interval == 0:
                    self.train_log(start_time, epoch, epochs, batch_i, n_batches, d_loss, g_loss)
                    print(g_loss)
            
            if epoch >= 0:
                t_loss = self.valid_cell(x_test_sim, y_test_sim)
                
                if self.method == "srgan":
                    self.valid_log_sr(start_time, epoch, epochs, batch_i, n_batches, d_loss, g_loss, t_loss)
                    valid_loss.append(t_loss[0])
                    valid_acc.append(t_loss[1])
                else:
                    self.valid_log(start_time, epoch, epochs, batch_i, n_batches, d_loss, g_loss, t_loss)
                    valid_loss.append(t_loss[2])
                    valid_acc.append(t_loss[4])
                
                train_loss.append(g_loss[2])
                train_acc.append(g_loss[4])
                
                waited = len(valid_loss) - 1 - np.argmin(valid_loss)
                print('waited for', waited, valid_loss)
                    
                if waited == 0:
                    self.generator.save(outPath + 'model_epoch'+ str(epoch) +'.h5')   
                        
                if waited > patience:
                    break
            
        return train_acc, train_loss, valid_acc, valid_loss
    
    def build_vgg(self):
        """
        Builds a pre-trained VGG19 model that outputs image features extracted at the
        third block of the model
        """
        vgg = VGG19(weights="imagenet")
        # Set outputs to outputs of last conv. layer in block 3
        # See architecture at: https://github.com/keras-team/keras/blob/master/keras/applications/vgg19.py
        vgg.outputs = [vgg.layers[9].output]

        img = Input(shape=self.img_shape)
        imgc = Concatenate()([img, img, img]) 

        # Extract image features
        img_features = vgg(imgc)

        return Model(img, img_features)

    
    def train_log(self, start_time, epoch, epochs, batch_i, n_batches, d_loss, g_loss):
        elapsed_time = datetime.datetime.now() - start_time
        print ("[Epoch %d/%d-%d/%d] [D loss&acc: %.3f, %.3f%%] [G loss&accA&accB: %.3f, %.3f%%, %.3f%%] time: %s" % (epoch, epochs,
                                                                                batch_i, n_batches,
                                                                                d_loss[0], 100*d_loss[1],
                                                                                100*g_loss[2], 100*g_loss[3], 100*g_loss[4],
                                                                                elapsed_time))    
    
    def valid_log_sr(self, start_time, epoch, epochs, batch_i, n_batches, d_loss, g_loss, t_loss):
        elapsed_time = datetime.datetime.now() - start_time
        print ("[Epoch %d/%d-%d/%d] [D loss&acc: %.3f, %.3f%%] [G loss&accA&accB: %.3f, %.3f%%, %.3f%%] time: %s" % (epoch, epochs,
                                                                                batch_i, n_batches,
                                                                                d_loss[0], 100*d_loss[1],
                                                                                100*g_loss[2], 100*g_loss[3], 100*g_loss[4],
                                                                                elapsed_time))   
        print(t_loss)
    
    def valid_log(self, start_time, epoch, epochs, batch_i, n_batches, d_loss, g_loss, t_loss):
        elapsed_time = datetime.datetime.now() - start_time
        print ("[Epoch %d/%d-%d/%d] [D loss&acc: %.3f, %.3f%%] [G loss&accA&accB: %.3f, %.3f%%, %.3f%%] [Test loss&acc: %.3f, %.3f%%, %.3f%%] time: %s" % (epoch, epochs,
                                                                                batch_i, n_batches,
                                                                                d_loss[0], 100*d_loss[1],
                                                                                100*g_loss[2], 100*g_loss[3], 100*g_loss[4],
                                                                                100*t_loss[2], 100*t_loss[3], 100*t_loss[4],
                                                                                elapsed_time))    

In [ ]:
# Order the image dimension acc. to TensorFlow (batc_hsize, rows, cols, channels)
K.set_image_dim_ordering('tf')

scale = 25
p_size_1 = 128 # Compared with 256, which larger may generate round corners
trainPath = r"../tmp_data/data_feng_/geb" + str(scale) +  "/"

# save image patch arrays
x_train_sim = np.load(trainPath + "x_train_sim.npy")
y_train_sim = np.load(trainPath + "y_train_sim.npy")
x_test_sim = np.load(trainPath + "x_test_sim.npy")
y_test_sim = np.load(trainPath + "y_test_sim.npy")

input_shape1 = (None, None, 1) #x_train_sim[0].shape
print('Input Shape of the trains', x_train_sim.shape)
print('Input Shape of the tests', x_test_sim.shape)

In [ ]:
def load_batch(x_train_sim, y_train_sim, batch_size):
    total_samples = len(x_train_sim)
    ids = np.arange(total_samples)
    np.random.shuffle(ids)
    n_batches = int(total_samples / batch_size)
    for i in range(n_batches-1):
        batch_idx = ids[i*batch_size:(i+1)*batch_size]
        imgs_A = x_train_sim[batch_idx]
        imgs_B = y_train_sim[batch_idx]
        yield imgs_B, imgs_A     
        
def load_data(x_test_sim, y_test_sim, batch_size=1):
    return x_test_sim  

def save_hist_local(train_acc, train_loss, valid_acc, valid_loss, outPath):    
    ### Save history
    History1_loss = train_loss
    History1_acc = train_acc
    History1_val_loss = valid_loss
    History1_val_acc = valid_acc

    thefile1 = open(outPath + 'History1_loss.txt', 'w')
    for item in History1_loss:
        thefile1.write("%s\n" % item)
    thefile1.close()

    thefile2 = open(outPath + 'History1_acc.txt', 'w')
    for item in History1_acc:
        thefile2.write("%s\n" % item)
    thefile2.close()

    thefile3 = open(outPath + 'History1_val_loss.txt', 'w')
    for item in History1_val_loss:
        thefile3.write("%s\n" % item)
    thefile3.close()

    thefile4 = open(outPath + 'History1_val_acc.txt', 'w')
    for item in History1_val_acc:
        thefile4.write("%s\n" % item)
    thefile4.close()

### Train residual unet + Simple

In [ ]:
############ Path Setting ##############
outPath = r"../tmp_results/predictions/"
timestr = strftime("U"+str(p_size_1)+"GAN_%Y-%m-%d %H-%M-%S", gmtime())
outPath = outPath + timestr + '_' + str(scale)+ "/"
check_and_create(outPath)
print(outPath)

In [ ]:
gan = GAN4MapGen("p2p", batch_size = 16)
train_acc, train_loss, valid_acc, valid_loss = gan.train(x_train_sim, y_train_sim, x_test_sim, y_test_sim, 
                                                         outPath, epochs=50, sample_interval=1000, patience = 4)

In [ ]:
save_hist_local(train_acc, train_loss, valid_acc, valid_loss, outPath)